In [1]:
import xml.etree.ElementTree as ET
import os

In [2]:
#See sample xml file for reference
def parse_annotation(ann_dir, img_dir, labels=[], x_ratio =1, y_ratio = 1):
    all_imgs = []
    seen_labels = {}
    
    for ann in sorted(os.listdir(ann_dir)):
        img = {'object':[]}

        tree = ET.parse(os.path.join(ann_dir, ann))
        
        for elem in tree.iter():
            if 'filename' in elem.tag:
                img['filename'] = os.path.join(img_dir, elem.text)
            if 'width' in elem.tag:
                img['width'] = int(elem.text)
            if 'height' in elem.tag:
                img['height'] = int(elem.text)
            if 'object' in elem.tag or 'part' in elem.tag:
                obj = {}
                
                for attr in list(elem):
                    if 'name' in attr.tag:
                        obj['name'] = attr.text

                        if obj['name'] in seen_labels:
                            seen_labels[obj['name']] += 1
                        else:
                            seen_labels[obj['name']] = 1
                        
                        if len(labels) > 0 and obj['name'] not in labels:
                            break
                        else:
                            img['object'] += [obj]
                            
                    if 'bndbox' in attr.tag:
                        for dim in list(attr):
                            if 'xmin' in dim.tag:
                                obj['xmin'] = int(round(float(dim.text))*x_ratio)
                            if 'ymin' in dim.tag:
                                obj['ymin'] = int(round(float(dim.text))*y_ratio)
                            if 'xmax' in dim.tag:
                                obj['xmax'] = int(round(float(dim.text))*x_ratio)
                            if 'ymax' in dim.tag:
                                obj['ymax'] = int(round(float(dim.text))*y_ratio)

        if len(img['object']) > 0:
            all_imgs += [img]
                        
    return all_imgs, seen_labels

In [49]:
LABELS = ['background','guinness','dhl','stellaartois','fosters','becks','nvidia','shell', 'corona', 'ford', 'google', 'aldi', 'pepsi', 'apple','fedex', 'starbucks', 'singha', 'heineken', 'carlsberg', 'paulaner', 'ferrari', 'ups', 'cocacola', 'erdinger', 'tsingtao', 'texaco', 'esso', 'chimay', 'bmw', 'adidas', 'rittersport', 'hp', 'milka', 'adidas_text', '3m', 'heineken_text', 'anz', 'anz_text', 'americanexpress', 'americanexpress_text', 'mastercard', 'visa', 'allianz_text', 'basf', 'audi', 'audi_text', 'volvo', 'mini', 'mercedesbenz', 'toyota', 'facebook', 'youtube', 'playstation', 'puma', 'intel', 'bosch_text', 'teslamotors', 'tnt', 'umbro', 'standard_liege', 'gucci', 'dexia', 'puma_text', 'nike', 'adidas1', 'base', 'citroen_text', 'total', 'bfgoodrich', 'kia', 'citroen', 'quick', 'carglass', 'airness', 'bik', 'reebok1', 'bridgestone_text', 'us_president', 'bridgestone', 'blackmores', 'cpa_australia', 'cvspharmacy', 'cvs', 'caterpillar', 'burgerking', 'nissan', 'amazon', 'fly_emirates', 'hh', 'head', 'head_text', 'heraldsun', 'hisense', 'ibm', 'ikea', 'jacobscreek', 'jurlique', 'loreal', 'lacoste', 'lacoste_text', 'mk', 'maxxis', 'mcdonalds', 'mcdonalds_text', 'mccafe', 'philips', 'burgerking_text', 'subway', 'medibank', 'nb', 'nike_text', 'optus', 'optus_yes', 'pampers', 'rolex', 'rolex_text', 'samsung', 't-mobile', 'android', 'at_and_t', 'zara', 'michelin', 'goodyear', 'walmart_text', 'walmart', 'coke', 'williamhill', 'yonex_text', 'yonex', 'abus' , 'accenture', 'airhawk', 'honda_text', 'aldi_text', 'target_text', 'alfaromeo', 'allett', 'allianz', 'aluratek', 'aluratek_text', 'amcrest', 'amcrest_text', 'apc', 'apecase', 'aquapac_text', 'aral', 'lotto', 'volkswagen', 'honda', 'ec', 'armani', 'armitron', 'aspirin', 'bayer', 'asus', 'athalon', 'mercedesbenz_text', 'lexus', 'axa', 'bacardi', 'bankofamerica', 'bankofamerica_text', 'kodak', 'barbie', 'barclays', 'batman', 'bbc', 'bbva', 'bellataylor', 'bellodigital', 'bellodigital_text', 'bem', 'twitter', 'benrus', 'bershka', 'bionade', 'blizzardentertainment', 'boeing', 'boeing_text', 'unicef', 'bosch', 'porsche', 'porsche_text', 'opel', 'siemens', 'panasonic', 'bottegaveneta', 'budweiser', 'budweiser_text', 'corona_text' , 'bulgari', 'pizzahut', 'costa', 'calvinklein', 'canon', 'sony', 'carters', 'cartier', 'chanel', 'chanel_text', 'prada', 'chevrolet', 'chevrolet_text', 'subaru', 'chevron', 'chickfila', 'chiquita', 'cisco', 'rbc', 'citi', 'coach', 'sprite', 'gap', 'hyundai', 'colgate', 'comedycentral', 'converse', 'costco', 'homedepot_text', 'danone', 'disney', 'drpepper', 'dunkindonuts', 'ebay', 'espn', 'esso_text', 'mobil', 'tigerwash', 'spar', 'spar_text', 'shell_text', 'renault', 'evernote', 'firefox', 'nbc', 'redbull', 'vodafone', 'yahoo', 'fritolay', 'fritos', 'cheetos', 'doritos', 'lays', 'ruffles', 'sunchips', 'tostitos', 'pepsi_text', 'lg', 'generalelectric', 'gildan', 'gillette', 'venus', 'hanes', 'hermes', 'hersheys', 'kitkat', 'reeses', 'hm', 'jcrew', 'homedepot', 'hsbc', 'hsbc_text', 'huawei_text', 'huawei', 'sap', 'hyundai_text', 'windows', 'internetexplorer', 'jackinthebox', 'jagermeister', 'johnnywalker', 'kelloggs', 'lego', 'kfc', 'suzuki', 'kraft', 'jello', 'maxwellhouse', 'miraclewhip', 'philadelphia', 'planters', 'velveeta', 'lamborghini', 'levis', 'lexus_text', 'londonunderground', 'lv', 'luxottica', 'marlboro', 'marlboro_text', 'marlboro_fig', 'maserati', 'microsoft', 'millerhighlife', 'mitsubishi', 'motorola', 'mtv', 'nasa', 'nescafe', 'netflix', 'nintendo', 'nissan_text', 'infiniti', 'infiniti_text', 'nivea', 'northface', 'obey', 'olympics', 'oracle', 'pepsi_text1', 'pizzahut_hut', 'poloralphlauren', 'recycling', 'redbull_text', 'firelli', 'santander_text', 'reebok_text', 'reebok', 'tommyhilfiger', 'republican', 'santander', 'schwinn', 'sega', 'shell_text1', 'select', 'skechers', 'vaio', 'soundcloud', 'soundrop', 'spiderman', 'superman', 'supreme', 'tacobell', 'target', 'toyota_text', 'thomsonreuters', 'timberland', 'tissot', 'verizon', 'scion_text', 'underarmour', 'uniqlo', 'uniqlo1', 'unitednations', 'verizon_text', 'volkswagen_text', 'wellsfargo','wellsfargo_text', 'warnerbros', 'wii', 'wordpress', 'xbox', 'yamaha']
# array containing labels. Can be more than one.
#train_image_folder = '/mnt/DABA7127BA7100F5/projects/data/pi_data'
#train_annot_folder = '/mnt/DABA7127BA7100F5/projects/data/pi_labels'
#C:\Users\Hammad\Pictures\openlogo\Annotations
train_annot_folder ="C:/Users/Hammad/Pictures/openlogo/Annotations"
train_image_folder = "C:/Users/Hammad/Pictures/openlogo/JPEGImages"



train_imgs, seen_train_labels = parse_annotation(train_annot_folder, train_image_folder, labels=LABELS,)

#label_ids = seen_train_labels.copy()
labels_ids = {'background': 0,'guinness': 1,'dhl': 2,'stellaartois': 3,'fosters': 4,'becks': 5,'nvidia': 6,'shell': 7, 'corona': 8, 'ford': 9, 'google': 10, 'aldi': 11, 'pepsi': 12, 'apple': 13, 'fedex': 14, 'starbucks': 15, 'singha': 16, 'heineken': 17, 'carlsberg': 18, 'paulaner': 19, 'ferrari': 20, 'ups': 21, 'cocacola': 22, 'erdinger': 23, 'tsingtao': 24, 'texaco': 25, 'esso': 26, 'chimay': 27, 'bmw': 28, 'adidas': 29, 'rittersport': 30, 'hp': 31, 'milka': 32, 'adidas_text': 33, '3m': 34, 'heineken_text': 35, 'anz': 36, 'anz_text': 37, 'americanexpress': 38, 'americanexpress_text': 39, 'mastercard': 40, 'visa': 41, 'allianz_text': 42, 'basf': 43, 'audi': 44, 'audi_text': 45, 'volvo': 46, 'mini': 47, 'mercedesbenz': 48, 'toyota': 49, 'facebook': 50, 'youtube': 51, 'playstation': 52, 'puma': 53, 'intel': 54, 'bosch_text': 55, 'teslamotors': 56, 'tnt': 57, 'umbro': 58, 'standard_liege': 59, 'gucci': 60, 'dexia': 61, 'puma_text': 62, 'nike': 63, 'adidas1': 64, 'base': 65, 'citroen_text': 66, 'total': 67, 'bfgoodrich': 68, 'kia': 69, 'citroen': 70, 'quick': 71, 'carglass': 72, 'airness': 73, 'bik': 74, 'reebok1': 75, 'bridgestone_text': 76, 'us_president': 77, 'bridgestone': 78, 'blackmores': 79, 'cpa_australia': 80, 'cvspharmacy': 81, 'cvs': 82, 'caterpillar': 83, 'burgerking': 84, 'nissan': 85, 'amazon': 86, 'fly_emirates': 87, 'hh': 88, 'head': 89, 'head_text': 90, 'heraldsun': 91, 'hisense': 92, 'ibm': 93, 'ikea': 94, 'jacobscreek': 95, 'jurlique': 96, 'loreal': 97, 'lacoste': 98, 'lacoste_text': 99, 'mk': 100, 'maxxis': 101, 'mcdonalds': 102, 'mcdonalds_text': 103, 'mccafe': 104, 'philips': 105, 'burgerking_text': 106, 'subway': 107, 'medibank': 108, 'nb': 109, 'nike_text': 110, 'optus': 111, 'optus_yes': 112, 'pampers': 113, 'rolex': 114, 'rolex_text': 115, 'samsung': 116, 't-mobile': 117, 'android': 118, 'at_and_t': 119, 'zara': 120, 'michelin': 121, 'goodyear': 122, 'walmart_text': 123, 'walmart': 124, 'coke': 125, 'williamhill': 126, 'yonex_text': 127, 'yonex': 128, 'abus': 129, 'accenture': 130, 'airhawk': 131, 'honda_text': 132, 'aldi_text': 133, 'target_text': 134, 'alfaromeo': 135, 'allett': 136, 'allianz': 137, 'aluratek': 138, 'aluratek_text': 139, 'amcrest': 140, 'amcrest_text': 141, 'apc': 142, 'apecase': 143, 'aquapac_text': 144, 'aral': 145, 'lotto': 146, 'volkswagen': 147, 'honda': 148, 'ec': 149, 'armani': 150, 'armitron': 151, 'aspirin': 152, 'bayer': 153, 'asus': 154, 'athalon': 155, 'mercedesbenz_text': 156, 'lexus': 157, 'axa': 158, 'bacardi': 159, 'bankofamerica': 160, 'bankofamerica_text': 161, 'kodak': 162, 'barbie': 163, 'barclays': 164, 'batman': 165, 'bbc': 166, 'bbva': 167, 'bellataylor': 168, 'bellodigital': 169, 'bellodigital_text': 170, 'bem': 171, 'twitter': 172, 'benrus': 173, 'bershka': 174, 'bionade': 175, 'blizzardentertainment': 176, 'boeing': 177, 'boeing_text': 178, 'unicef': 179, 'bosch': 180, 'porsche': 181, 'porsche_text': 182, 'opel': 183, 'siemens': 184, 'panasonic': 185, 'bottegaveneta': 186, 'budweiser': 187, 'budweiser_text': 188, 'corona_text': 189, 'bulgari': 190, 'pizzahut': 191, 'costa': 192, 'calvinklein': 193, 'canon': 194, 'sony': 195, 'carters': 196, 'cartier': 197, 'chanel': 198, 'chanel_text': 199, 'prada': 200, 'chevrolet': 201, 'chevrolet_text': 202, 'subaru': 203, 'chevron': 204, 'chickfila': 205, 'chiquita': 206, 'cisco': 207, 'rbc': 208, 'citi': 209, 'coach': 210, 'sprite': 211, 'gap': 212, 'hyundai': 213, 'colgate': 214, 'comedycentral': 215, 'converse': 216, 'costco': 217, 'homedepot_text': 218, 'danone': 219, 'disney': 220, 'drpepper': 221, 'dunkindonuts': 222, 'ebay': 223, 'espn': 224, 'esso_text': 225, 'mobil': 226, 'tigerwash': 227, 'spar': 228, 'spar_text': 229, 'shell_text': 230, 'renault': 231, 'evernote': 232, 'firefox': 233, 'nbc': 234, 'redbull': 235, 'vodafone': 236, 'yahoo': 237, 'fritolay': 238, 'fritos': 239, 'cheetos': 240, 'doritos': 241, 'lays': 242, 'ruffles': 243, 'sunchips': 244, 'tostitos': 245, 'pepsi_text': 246, 'lg': 247, 'generalelectric': 248, 'gildan': 249, 'gillette': 250, 'venus': 251, 'hanes': 252, 'hermes': 253, 'hersheys': 254, 'kitkat': 255, 'reeses': 256, 'hm': 257, 'jcrew': 258, 'homedepot': 259, 'hsbc': 260, 'hsbc_text': 261, 'huawei_text': 262, 'huawei': 263, 'sap': 264, 'hyundai_text': 265, 'windows': 266, 'internetexplorer': 267, 'jackinthebox': 268, 'jagermeister': 269, 'johnnywalker': 270, 'kelloggs': 271, 'lego': 272, 'kfc': 273, 'suzuki': 274, 'kraft': 275, 'jello': 276, 'maxwellhouse': 277, 'miraclewhip': 278, 'philadelphia': 279, 'planters': 280, 'velveeta': 281, 'lamborghini': 282, 'levis': 283, 'lexus_text': 284, 'londonunderground': 285, 'lv': 286, 'luxottica': 287, 'marlboro': 288, 'marlboro_text': 289, 'marlboro_fig': 290, 'maserati': 291, 'microsoft': 292, 'millerhighlife': 293, 'mitsubishi': 294, 'motorola': 295, 'mtv': 296, 'nasa': 297, 'nescafe': 298, 'netflix': 299, 'nintendo': 300, 'nissan_text': 301, 'infiniti': 302, 'infiniti_text': 303, 'nivea': 304, 'northface': 305, 'obey': 306, 'olympics': 307, 'oracle': 308, 'pepsi_text1': 309, 'pizzahut_hut': 310, 'poloralphlauren': 311, 'recycling': 312, 'redbull_text': 313, 'firelli': 314, 'santander_text': 315, 'reebok_text': 316, 'reebok': 317, 'tommyhilfiger': 318, 'republican': 319, 'santander': 320, 'schwinn': 321, 'sega': 322, 'shell_text1': 323, 'select': 324, 'skechers': 325, 'vaio': 326, 'soundcloud': 327, 'soundrop': 328, 'spiderman': 329, 'superman': 330, 'supreme': 331, 'tacobell': 332, 'target': 333, 'toyota_text': 334, 'thomsonreuters': 335, 'timberland': 336, 'tissot': 337, 'verizon': 338, 'scion_text': 339, 'underarmour': 340, 'uniqlo': 341, 'uniqlo1': 342, 'unitednations': 343, 'verizon_text': 344, 'volkswagen_text': 345, 'warnerbros': 346,'wellsfargo': 347, 'wellsfargo_text': 348, 'wii': 349, 'wordpress': 350, 'xbox': 351, 'yamaha': 352}

In [53]:
train_imgs #Checking data format

[{'object': [{'name': 'guinness',
    'xmin': 333,
    'ymin': 238,
    'xmax': 595,
    'ymax': 414}],
  'filename': 'C:/Users/Hammad/Pictures/openlogo/JPEGImages\\1008198576.jpg',
  'width': 683,
  'height': 1024},
 {'object': [{'name': 'dhl',
    'xmin': 503,
    'ymin': 122,
    'xmax': 1022,
    'ymax': 387}],
  'filename': 'C:/Users/Hammad/Pictures/openlogo/JPEGImages\\1016381746.jpg',
  'width': 1024,
  'height': 685},
 {'object': [{'name': 'stellaartois',
    'xmin': 52,
    'ymin': 25,
    'xmax': 978,
    'ymax': 1022}],
  'filename': 'C:/Users/Hammad/Pictures/openlogo/JPEGImages\\102448630.jpg',
  'width': 1024,
  'height': 1024},
 {'object': [{'name': 'fosters',
    'xmin': 423,
    'ymin': 164,
    'xmax': 586,
    'ymax': 340}],
  'filename': 'C:/Users/Hammad/Pictures/openlogo/JPEGImages\\1030265231.jpg',
  'width': 1024,
  'height': 684},
 {'object': [{'name': 'fosters',
    'xmin': 391,
    'ymin': 448,
    'xmax': 775,
    'ymax': 875}],
  'filename': 'C:/Users/Hammad/

In [54]:
f = open ('pi_labels.txt', 'w+')
for img_number in range(0,len(train_imgs)):
    image = train_imgs[img_number]
    im_name = image['filename'].split('/')[-1]
    objects = image['object']
    # print (im_name)
    line = im_name
    for objs in objects:
        xmin = objs['xmin']
        ymin = objs['ymin']
        xmax = objs['xmax']
        ymax = objs['ymax']
        c_id = labels_ids[objs['name']]
#         print (xmin, ymin, xmax, ymax, c_id)
        line += ' '+str(xmin)+',' +str(ymin)+',' +str(xmax)+','+str(ymax)+','+ str(c_id)

#     print (line)
    f.write (line + '\n')
f.close()